In [1]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Real
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import time

In [2]:
number_of_calls = 400
number_of_random_start = 250
rbf_lengthscale = 0.08

delta = 0.3

n_jobs = -1 # Use all available processors

In [3]:
# Define the objective function to maximize the surrogate model
def objective_function(x):
    # Predict using the surrogate model
    y_pred, _ = surrogate_model.predict(np.array(x).reshape(1, -1), return_std=True)
    return -y_pred[0]  # Negate to maximize

In [4]:
def find_dimensions_around_max(X, Y):
    max_index = np.argmax(Y)
    input_column = X[max_index]
    dimensions = [Real(max(0.000000, x - delta), min(0.999999, x + delta)) for x in input_column]
    return dimensions

In [5]:
def find_dimensions_around_local_max(X, Y):
    max_index = np.argmax(Y)
    
    # Identify potential local maxima
    potential_maxima = [i for i in range(len(Y)) if Y[i] > 0.8 * Y[max_index]] 
    
    # Choose the local maximum with X values far enough from the absolute maximum
    def distance_from_absolute_max(i):
        return np.max(np.abs(X[i] - X[max_index]))

    local_max_index = max(potential_maxima, key=distance_from_absolute_max)

    input_column = X[local_max_index]
    dimensions = [Real(max(0.000000, x - delta), min(0.999999, x + delta)) for x in input_column]
    return dimensions

Week 1: random manual numbers

Week 2: trying linear regression vs forrest + manual maxima

Week 3: trying linear regression + change parameter + manual maxima

Week 4: Current bayesian implementation
    trying rbf kernel
    
week 5:
    trying matern kernel
    
week 6:
    trying sum kernel
    
week 7: Trying the most succesfull week (rbf) so far, with additional data, on the entirety of 0 to 1
    number_of_calls = 150
    number_of_random_start = 75
    rbf_lengthscale = 0.08
    
week 8: Only predicting around the highest values for each set instead of 0.0 to 9.999

Week 9: Same as week 8 with different RBF lengthscale

Week 10: Trying to look away from the current maximum as we could potentially be stuck in a local max. Finding maximum far away from the max and look around it.

Week 11: For the final submission, take the best of the above and parallelize the code to run on a better machine with a much higher number of calls and random starts

***
## Function 1

In [6]:
f1_X = np.load('initial_data/function_1/initial_inputs.npy')
f1_Y = np.load('initial_data/function_1/initial_outputs.npy')

new_f1_X = np.load('initial_data2/function_1/initial_inputs.npy')
new_f1_Y = np.load('initial_data2/function_1/initial_outputs.npy')

# Concatenate the new data with the original data
f1_X = np.concatenate((f1_X, new_f1_X), axis=0)
f1_Y = np.concatenate((f1_Y, new_f1_Y), axis=0)

#Week 1
f1_X = np.vstack((f1_X, np.array([0.83, 0.29])))
f1_Y = np.append(f1_Y, -3.2860097425164E-109)
#Week 2
f1_X = np.vstack((f1_X, np.array([0.619903,0.245888])))
f1_Y = np.append(f1_Y, -1.01694853412876E-50)
#Week 3
f1_X = np.vstack((f1_X, np.array([0.775021,0.265669])))
f1_Y = np.append(f1_Y, -1.24778222636534E-106)
#Week 4
f1_X = np.vstack((f1_X, np.array([0.549448,0.600462])))
f1_Y = np.append(f1_Y, -0.0000110332053603618)
#Week 5
f1_X = np.vstack((f1_X, np.array([0.785116,0.729309])))
f1_Y = np.append(f1_Y, -1.02E-24)
#Week 6
f1_X = np.vstack((f1_X, np.array([0.743678,0.764168])))
f1_Y = np.append(f1_Y, -1.41453726950899E-23)
#Week 7
f1_X = np.vstack((f1_X, np.array([0.597106,0.614121])))
f1_Y = np.append(f1_Y, 0.0762719854223018)
#Week 8
f1_X = np.vstack((f1_X, np.array([0.614246,0.615296])))
f1_Y = np.append(f1_Y, 0.790547512875297)
#Week 9
f1_X = np.vstack((f1_X, np.array([0.677835,0.593512])))
f1_Y = np.append(f1_Y, -0.0003792444576)

In [7]:
kernel = RBF(length_scale=rbf_lengthscale, length_scale_bounds='fixed')
surrogate_model = make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=kernel))

#kernel = Matern(length_scale=1.0, nu=2.5)
#surrogate_model = GaussianProcessRegressor(kernel=kernel)
#surrogate_model = make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=kernel))

surrogate_model.fit(f1_X, f1_Y)

dimensions = find_dimensions_around_max(f1_X, f1_Y)
#dimensions = [
#    Real(0.000000, 0.999999, name='x1'),
#    Real(0.000000, 0.999999, name='x2')
#]

start_time = time.time()

result = gp_minimize(
    objective_function,
    dimensions=dimensions,
    n_calls=number_of_calls, 
    n_random_starts=number_of_random_start,
    n_jobs=n_jobs,
)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

print("Input: {} --> Output: {}".format(result.x,-result.fun))
formatted_x1 = '-'.join([f'{x:.6f}' for x in result.x])
print(formatted_x1)

Elapsed time: 10441.67767572403 seconds
Input: [0.6255801946376838, 0.6160773268517411] --> Output: 1.0793423239757796
0.625580-0.616077


***
## Function 2

In [8]:
f2_X = np.load('initial_data/function_2/initial_inputs.npy')
f2_Y = np.load('initial_data/function_2/initial_outputs.npy')

new_f2_X = np.load('initial_data2/function_2/initial_inputs.npy')
new_f2_Y = np.load('initial_data2/function_2/initial_outputs.npy')

# Concatenate the new data with the original data
f2_X = np.concatenate((f2_X, new_f2_X), axis=0)
f2_Y = np.concatenate((f2_Y, new_f2_Y), axis=0)

#Week 1
f2_X = np.vstack((f2_X, np.array([0.71, 0.99])))
f2_Y = np.append(f2_Y, 0.7093867276)
#Week 2
f2_X = np.vstack((f2_X, np.array([0.87779,0.99])))
f2_Y = np.append(f2_Y, -0.02792512905)
#Week 3
f2_X = np.vstack((f2_X, np.array([0.877791,0.99])))
f2_Y = np.append(f2_Y, 0.07256572223)
#Week 4
f2_X = np.vstack((f2_X, np.array([0.883564,0.99])))
f2_Y = np.append(f2_Y, 0.009198636598)
#Week 5
f2_X = np.vstack((f2_X, np.array([0.913759,0.989874])))
f2_Y = np.append(f2_Y, -0.1076669936)
#Week 6
f2_X = np.vstack((f2_X, np.array([0.974913,0.999999])))
f2_Y = np.append(f2_Y, -0.0396112707405163)
#Week 7
f2_X = np.vstack((f2_X, np.array([0.931841,0.990063])))
f2_Y = np.append(f2_Y, 0.01882726689)
#Week 8
f2_X = np.vstack((f2_X, np.array([0.709824,0.986201])))
f2_Y = np.append(f2_Y, 0.6477917306)
#Week 9
f2_X = np.vstack((f2_X, np.array([0.64625,0.999999])))
f2_Y = np.append(f2_Y, 0.3140346957)

In [9]:
kernel = RBF(length_scale=rbf_lengthscale, length_scale_bounds='fixed')
surrogate_model = make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=kernel))

surrogate_model.fit(f2_X, f2_Y)

dimensions = find_dimensions_around_max(f2_X, f2_Y)

#dimensions = [
#    Real(0.000000, 0.999999, name='x1'),
#    Real(0.000000, 0.999999, name='x2')
#]

result = gp_minimize(
    objective_function,
    dimensions=dimensions,
    n_calls=number_of_calls, 
    verbose=False,
    n_random_starts=number_of_random_start,
    n_jobs=n_jobs,
)

print("Input: {} --> Output: {}".format(result.x,-result.fun))
formatted_x2 = '-'.join([f'{x:.6f}' for x in result.x])
print(formatted_x2)

Input: [0.9225606999716331, 0.9873649261924347] --> Output: 185.50617078349933
0.922561-0.987365


***
## Function 3

In [10]:
f3_X = np.load('initial_data/function_3/initial_inputs.npy')
f3_Y = np.load('initial_data/function_3/initial_outputs.npy')

new_f3_X = np.load('initial_data2/function_3/initial_inputs.npy')
new_f3_Y = np.load('initial_data2/function_3/initial_outputs.npy')

# Concatenate the new data with the original data
f3_X = np.concatenate((f3_X, new_f3_X), axis=0)
f3_Y = np.concatenate((f3_Y, new_f3_Y), axis=0)

#Week 1
f3_X = np.vstack((f3_X, np.array([0.54,0.65,0.2])))
f3_Y = np.append(f3_Y, -0.1309050532)
#Week 2
f3_X = np.vstack((f3_X, np.array([0.718668,0.999945,0.272151])))
f3_Y = np.append(f3_Y, -0.1413686626)
#Week 3
f3_X = np.vstack((f3_X, np.array([0.596189,0.614615,0.214115])))
f3_Y = np.append(f3_Y, -0.11486006842883)
#Week 4
f3_X = np.vstack((f3_X, np.array([0.124154,0.849855,0.738203])))
f3_Y = np.append(f3_Y, -0.142476683810238)
#Week 5
f3_X = np.vstack((f3_X, np.array([0.91942,0.037235,0.806635])))
f3_Y = np.append(f3_Y, -0.1409064565)
#Week 6
f3_X = np.vstack((f3_X, np.array([[0.56002,0.999999,0.999999]])))
f3_Y = np.append(f3_Y, -0.4871510136)
#Week 7
f3_X = np.vstack((f3_X, np.array([0.999999,0.729995,0.00984 ])))
f3_Y = np.append(f3_Y, -0.09678844518)
#Week 8
f3_X = np.vstack((f3_X, np.array([[0.475798,0.222853,0.434177]])))
f3_Y = np.append(f3_Y, -0.05597062399)
#Week 9
# Same input ...

In [11]:
kernel = RBF(length_scale=rbf_lengthscale, length_scale_bounds='fixed')
surrogate_model = make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=kernel))

surrogate_model.fit(f3_X, f3_Y)

dimensions = find_dimensions_around_max(f3_X, f3_Y)

#dimensions = [
#    Real(0.000000, 0.999999, name='x1'), 
#    Real(0.000000, 0.999999, name='x2'),
#    Real(0.000000, 0.999999, name='x3') 
#]

result = gp_minimize(
    objective_function,
    dimensions=dimensions,
    n_calls=number_of_calls,
    verbose=False,
    n_random_starts=number_of_random_start,
    n_jobs=n_jobs,
)

print("Input: {} --> Output: {}".format(result.x,-result.fun))
formatted_x3 = '-'.join([f'{x:.6f}' for x in result.x])
print(formatted_x3)

Input: [0.7836074381413918, 0.022852844175113518, 0.43287507074837694] --> Output: -1.3704923619454252e-52
0.783607-0.022853-0.432875


***
## Function 4

In [12]:
f4_X = np.load('initial_data/function_4/initial_inputs.npy')
f4_Y = np.load('initial_data/function_4/initial_outputs.npy')

new_f4_X = np.load('initial_data2/function_4/initial_inputs.npy')
new_f4_Y = np.load('initial_data2/function_4/initial_outputs.npy')

# Concatenate the new data with the original data
f4_X = np.concatenate((f4_X, new_f4_X), axis=0)
f4_Y = np.concatenate((f4_Y, new_f4_Y), axis=0)

#Week 1
f4_X = np.vstack((f4_X, np.array([0.12,0.101,0.36,0.24])))
f4_Y = np.append(f4_Y, -10.2679334484568)
#Week 2
f4_X = np.vstack((f4_X, np.array([0.401067,0.465459,0.990034,0.337921])))
f4_Y = np.append(f4_Y, -17.45206644)
#Week 3
f4_X = np.vstack((f4_X, np.array([0.398924,0.448872,0.912348,0.328706])))
f4_Y = np.append(f4_Y, -14.27743893)
#Week 4
f4_X = np.vstack((f4_X, np.array([0.961534,0.902131,0.835173,0.600658])))
f4_Y = np.append(f4_Y, -33.189198584856)
#Week 5
f4_X = np.vstack((f4_X, np.array([0.643583,0.999999,0.999999,0.033833])))
f4_Y = np.append(f4_Y, -37.2036304)
#Week 6
f4_X = np.vstack((f4_X, np.array([0.999999,0.048693,0.059318,0.293845])))
f4_Y = np.append(f4_Y, -25.7829225)
#Week 7
f4_X = np.vstack((f4_X, np.array([0.022451,0.9303,0.987882,0.003071])))
f4_Y = np.append(f4_Y, -36.95741928)
#Week 8
f4_X = np.vstack((f4_X, np.array([0.667426,0.339794,0.519347,0.168577])))
f4_Y = np.append(f4_Y, -8.40414859)
#Week 9
f4_X = np.vstack((f4_X, np.array([0.477766,0.328772,0.525826,0.349007])))
f4_Y = np.append(f4_Y, -2.353407563)

In [13]:
kernel = RBF(length_scale=rbf_lengthscale, length_scale_bounds='fixed')
surrogate_model = make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=kernel))

surrogate_model.fit(f4_X, f4_Y)

dimensions = find_dimensions_around_max(f4_X, f4_Y)

#dimensions = [
#    Real(0.000000, 0.999999, name='x1'),
#    Real(0.000000, 0.999999, name='x2'),
#    Real(0.000000, 0.999999, name='x3'),
#    Real(0.000000, 0.999999, name='x4')
#]

result = gp_minimize(
    objective_function,
    dimensions=dimensions,
    n_calls=number_of_calls, 
    verbose=False,
    n_random_starts=number_of_random_start,
    n_jobs=n_jobs,
)

print("Input: {} --> Output: {}".format(result.x,-result.fun))
formatted_x4 = '-'.join([f'{x:.6f}' for x in result.x])
print(formatted_x4)

Input: [0.32383136624676534, 0.4131185462839491, 0.6397357884770847, 0.5878370395471002] --> Output: -7.209599189149367e-44
0.323831-0.413119-0.639736-0.587837


***
## Function 5

In [14]:
f5_X = np.load('initial_data/function_5/initial_inputs.npy')
f5_Y = np.load('initial_data/function_5/initial_outputs.npy')

new_f5_X = np.load('initial_data2/function_5/initial_inputs.npy')
new_f5_Y = np.load('initial_data2/function_5/initial_outputs.npy')

# Concatenate the new data with the original data
f5_X = np.concatenate((f5_X, new_f5_X), axis=0)
f5_Y = np.concatenate((f5_Y, new_f5_Y), axis=0)

#Week 1
f5_X = np.vstack((f5_X, np.array([0.23,0.9,0.8,0.9])))
f5_Y = np.append(f5_Y, 1113.607380032)
#Week 2
f5_X = np.vstack((f5_X, np.array([0.119879,0.9,0.879484,0.957644])))
f5_Y = np.append(f5_Y, 1854.055026)
#Week 3
#Same input
#Week 4
f5_X = np.vstack((f5_X, np.array([0.781781,0.621022,0.901272,0.827289])))
f5_Y = np.append(f5_Y, 1000.424187)
#Week 5
f5_X = np.vstack((f5_X, np.array([0.510993,0.818001,0.7287,0.112178])))
f5_Y = np.append(f5_Y, 79.72680503)
#Week 6
f5_X = np.vstack((f5_X, np.array([0.221762,0.875412,0.838642,0.894668])))
f5_Y = np.append(f5_Y, 1115.020253)
#Week 7
f5_X = np.vstack((f5_X, np.array([0.439012,0.774128,0.377988,0.933526])))
f5_Y = np.append(f5_Y, 355.3160089)
#Week 8
f5_X = np.vstack((f5_X, np.array([0.30559,0.93653,0.95385,0.883531])))
f5_Y = np.append(f5_Y, 2136.283243)
#Week 9
f5_X = np.vstack((f5_X, np.array([0.298797,0.930974,0.948793,0.883932])))
f5_Y = np.append(f5_Y, 2052.197033)

In [15]:
kernel = RBF(length_scale=rbf_lengthscale, length_scale_bounds='fixed')
surrogate_model = make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=kernel))

surrogate_model.fit(f5_X, f5_Y)

dimensions = find_dimensions_around_max(f5_X, f5_Y)

#dimensions = [
#    Real(0.000000, 0.999999, name='x1'),
#    Real(0.000000, 0.999999, name='x2'),
#    Real(0.000000, 0.999999, name='x3'),
#    Real(0.000000, 0.999999, name='x4')
#]

result = gp_minimize(
    objective_function,
    dimensions=dimensions,
    n_calls=number_of_calls, 
    verbose=False,
    n_random_starts=number_of_random_start,
    n_jobs=n_jobs,
)

print("Input: {} --> Output: {}".format(result.x,-result.fun))
formatted_x5 = '-'.join([f'{x:.6f}' for x in result.x])
print(formatted_x5)

Input: [0.11989137917271951, 0.8625948936079095, 0.654016918680242, 0.8497836986528191] --> Output: 386.4463909784588
0.119891-0.862595-0.654017-0.849784


***
## Function 6

In [16]:
f6_X = np.load('initial_data/function_6/initial_inputs.npy')
f6_Y = np.load('initial_data/function_6/initial_outputs.npy')

new_f6_X = np.load('initial_data2/function_6/initial_inputs.npy')
new_f6_Y = np.load('initial_data2/function_6/initial_outputs.npy')

# Concatenate the new data with the original data
f6_X = np.concatenate((f6_X, new_f6_X), axis=0)
f6_Y = np.concatenate((f6_Y, new_f6_Y), axis=0)

#Week 1
f6_X = np.vstack((f6_X, np.array([0.71,0.16,0.75,0.67,0.09])))
f6_Y = np.append(f6_Y, -0.7309860846)
#Week 2
f6_X = np.vstack((f6_X, np.array([0.237921,0.191643,0.308479,0.169956,0.1230565])))
f6_Y = np.append(f6_Y, -1.345672776)
#Week 3
f6_X = np.vstack((f6_X, np.array([0.291906,0.224345,0.332472,0.174037,0.1414308])))
f6_Y = np.append(f6_Y, -1.33008597)
#Week 4
f6_X = np.vstack((f6_X, np.array([0.087971,0.967504,0.160638,0.054875,0.927092])))
f6_Y = np.append(f6_Y, -2.894154801)
#Week 5
f6_X = np.vstack((f6_X, np.array([0.041704,0.955936,0.122085,0.061743,0.014749])))
f6_Y = np.append(f6_Y, -2.329042984)
#Week 6
f6_X = np.vstack((f6_X, np.array([0.819207,0.121557,0.033515,0.000000,0.999999])))
f6_Y = np.append(f6_Y, -2.917421483)
#Week 7
f6_X = np.vstack((f6_X, np.array([0.810012,0.181364,0.961092,0.452903,0.954819])))
f6_Y = np.append(f6_Y, -1.966653483)
#Week 8
f6_X = np.vstack((f6_X, np.array([0.810125,0.054693,0.632552,0.603922,0.061692])))
f6_Y = np.append(f6_Y, -0.880088081054918)
#Week 9
f6_X = np.vstack((f6_X, np.array([0.828186,0.254693,0.632552,0.793997,0.156401])))
f6_Y = np.append(f6_Y, -0.534795741)

In [17]:
kernel = RBF(length_scale=rbf_lengthscale, length_scale_bounds='fixed')
surrogate_model = make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=kernel))

surrogate_model.fit(f6_X, f6_Y)

dimensions = find_dimensions_around_max(f6_X, f6_Y)

#dimensions = [
#    Real(0.000000, 0.999999, name='x1'),
#    Real(0.000000, 0.999999, name='x2'),
#    Real(0.000000, 0.999999, name='x3'),
#    Real(0.000000, 0.999999, name='x4'),
#    Real(0.000000, 0.999999, name='x5')
#]

result = gp_minimize(
    objective_function,
    dimensions=dimensions,
    n_calls=number_of_calls, 
    verbose=False,
    n_random_starts=number_of_random_start,
    n_jobs=n_jobs,
)

print("Input: {} --> Output: {}".format(result.x,-result.fun))
formatted_x6 = '-'.join([f'{x:.6f}' for x in result.x])
print(formatted_x6)

Input: [0.5952377402374274, 0.5208532556978974, 0.8813661264548225, 0.8861377712223617, 0.43024189463684104] --> Output: -6.861047891416782e-86
0.595238-0.520853-0.881366-0.886138-0.430242


***
## Function 7

In [18]:
f7_X = np.load('initial_data/function_7/initial_inputs.npy')
f7_Y = np.load('initial_data/function_7/initial_outputs.npy')

new_f7_X = np.load('initial_data2/function_7/initial_inputs.npy')
new_f7_Y = np.load('initial_data2/function_7/initial_outputs.npy')

# Concatenate the new data with the original data
f7_X = np.concatenate((f7_X, new_f7_X), axis=0)
f7_Y = np.concatenate((f7_Y, new_f7_Y), axis=0)

#Week 1
f7_X = np.vstack((f7_X, np.array([0.25,0.33,0.85,0.83,0.2,0.75])))
f7_Y = np.append(f7_Y, 0.50227388554458)
#Week 2
f7_X = np.vstack((f7_X, np.array([0.057896,0.924694,0.924571,0.073659,0.014944,0.951014])))
f7_Y = np.append(f7_Y, 0.0850275940993777)
#Week 3
f7_X = np.vstack((f7_X, np.array([0.067896,0.824694,0.724571,0.063659,0.034944,0.851014])))
f7_Y = np.append(f7_Y, 0.218179252964693)
#Week 4
f7_X = np.vstack((f7_X, np.array([0.822203,0.905431,0.608675,0.131364,0.711093,0.392675])))
f7_Y = np.append(f7_Y, 0.008208888021)
#Week 5
f7_X = np.vstack((f7_X, np.array([0.920947,0.488113,0.196897,0.978698,0.051959,0.620322])))
f7_Y = np.append(f7_Y, 0.001484417237)
#Week 6
f7_X = np.vstack((f7_X, np.array([0.287303,0.314722,0.926001,0.82834,0.146134,0.740828])))
f7_Y = np.append(f7_Y, 0.4076582295)
#Week 7
f7_X = np.vstack((f7_X, np.array([0.245841,0.958549,0.260444,0.898014,0.493964,0.432611])))
f7_Y = np.append(f7_Y, 0.08020004155)
#Week 8
f7_X = np.vstack((f7_X, np.array([0.191129,0.252443,0.391144,0.229153,0.363772,0.861907])))
f7_Y = np.append(f7_Y, 2.11378971012324)
#Week 9
f7_X = np.vstack((f7_X, np.array([0.191251,0.25412,0.390648,0.233326,0.362611,0.862228])))
f7_Y = np.append(f7_Y, 2.120640397)

In [19]:
kernel = RBF(length_scale=rbf_lengthscale, length_scale_bounds='fixed')
surrogate_model = make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=kernel))

surrogate_model.fit(f7_X, f7_Y)

dimensions = find_dimensions_around_max(f7_X, f7_Y)

#dimensions = [
#    Real(0.000000, 0.999999, name='x1'),
#    Real(0.000000, 0.999999, name='x2'),
#    Real(0.000000, 0.999999, name='x3'),
#    Real(0.000000, 0.999999, name='x4'),
#    Real(0.000000, 0.999999, name='x5'),
#    Real(0.000000, 0.999999, name='x6')
#]

result = gp_minimize(
    objective_function,
    dimensions=dimensions,
    n_calls=number_of_calls, 
    verbose=False,
    n_random_starts=number_of_random_start,
    n_jobs=n_jobs,
)

print("Input: {} --> Output: {}".format(result.x,-result.fun))
formatted_x7 = '-'.join([f'{x:.6f}' for x in result.x])
print(formatted_x7)

Input: [0.25827770422382823, 0.19498208768729175, 0.37756943444661667, 0.26768698477669683, 0.3204322419562342, 0.8389904491456184] --> Output: 2.6344275658812322e-05
0.258278-0.194982-0.377569-0.267687-0.320432-0.838990


***
## Function 8

In [20]:
f8_X = np.load('initial_data/function_8/initial_inputs.npy')
f8_Y = np.load('initial_data/function_8/initial_outputs.npy')

new_f8_X = np.load('initial_data2/function_8/initial_inputs.npy')
new_f8_Y = np.load('initial_data2/function_8/initial_outputs.npy')

# Concatenate the new data with the original data
f8_X = np.concatenate((f8_X, new_f8_X), axis=0)
f8_Y = np.concatenate((f8_Y, new_f8_Y), axis=0)

#Week 1
f8_X = np.vstack((f8_X, np.array([0.2,0.4,0.235,0.625,0.12,0.21,0.38,0.35])))
f8_Y = np.append(f8_Y, 9.27245)
#Week 2
f8_X = np.vstack((f8_X, np.array([0.009077,0.003419,0.022929,0.009043,0.009649,0.990244,0.035909,0.988755])))
f8_Y = np.append(f8_Y, 9.286087265)
#Week 3
#Same input
#Week 4
f8_X = np.vstack((f8_X, np.array([0.092456,0.271979,0.044189,0.249986,0.53335,0.540113,0.19916,0.544261])))
f8_Y = np.append(f8_Y, 9.915447257)
#Week 5
f8_X = np.vstack((f8_X, np.array([0.759009,0.486382,0.18137,0.476835,0.742339,0.499103,0.721871,0.420298])))
f8_Y = np.append(f8_Y, 8.353932516)
#Week 6
f8_X = np.vstack((f8_X, np.array([0.382459,0.730594,0.947017,0.031305,0.500582,0.735707,0.691031,0.037992])))
f8_Y = np.append(f8_Y, 6.872514052)
#Week 7
f8_X = np.vstack((f8_X, np.array([0.126204,0.631699,0.715524,0.975715,0.889768,0.526422,0.7435,0.64778])))
f8_Y = np.append(f8_Y, 7.460532387)
#Week 8
f8_X = np.vstack((f8_X, np.array([0.100398,0.273608,0.045201,0.252195,0.539865,0.547434,0.220625,0.544195])))
f8_Y = np.append(f8_Y, 9.915457022)
#Week 9
f8_X = np.vstack((f8_X, np.array([0.095536,0.278087,0.06115,0.257399,0.512098,0.521656,0.193101,0.53147 ])))
f8_Y = np.append(f8_Y, 9.915320762)

In [21]:
kernel = RBF(length_scale=rbf_lengthscale, length_scale_bounds='fixed')
surrogate_model = make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=kernel))

surrogate_model.fit(f8_X, f8_Y)

dimensions = find_dimensions_around_max(f8_X, f8_Y)

#dimensions = [
#    Real(0.000000, 0.999999, name='x1'),
#    Real(0.000000, 0.999999, name='x2'),
#    Real(0.000000, 0.999999, name='x3'),
#    Real(0.000000, 0.999999, name='x4'),
#    Real(0.000000, 0.999999, name='x5'),
#    Real(0.000000, 0.999999, name='x6'),
#    Real(0.000000, 0.999999, name='x7'),
#    Real(0.000000, 0.999999, name='x8')
#]

result = gp_minimize(
    objective_function,
    dimensions=dimensions,
    n_calls=number_of_calls, 
    verbose=False,
    n_random_starts=number_of_random_start,
    n_jobs=n_jobs,
)

print("Input: {} --> Output: {}".format(result.x,-result.fun))
formatted_x8 = '-'.join([f'{x:.6f}' for x in result.x])
print(formatted_x8)

Input: [0.012866824482350504, 0.3425759187623957, 0.03483457844239992, 0.19382159279286487, 0.47985148974167624, 0.5222622190565103, 0.22928933591860512, 0.45902693440390807] --> Output: 3.1291063429273053e-09
0.012867-0.342576-0.034835-0.193822-0.479851-0.522262-0.229289-0.459027
